In [ ]:
%pip install pandas numpy scikit-learn ucimlrepo lime bayesian-optimization shap

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.svm import SVC

from ucimlrepo import fetch_ucirepo 
import lime
from lime import lime_tabular

import pickle as pkl

from bayes_opt import BayesianOptimization
from tqdm import tqdm
import matplotlib.pyplot as plt

from scipy.stats import uniform, norm
from scipy.special import softmax
import shap

from local2global import Local2GlobalExplainer

MODEL_NAME = 'Wine'

MODEL_FUNCTION = DecisionTreeClassifier
model_params = {'random_state': 1340304}

coding = {"Wine": 109, 'German Credit': 144}


In [ ]:
# fetch dataset 
split_size = 0.3
seed = 1234567

dataset_id = coding[MODEL_NAME]

wine = fetch_ucirepo(id=109) 

In [ ]:

# data (as pandas dataframes) 
X = wine.data.features
y = wine.data.targets.to_numpy()
y = y.reshape((len(y), ))
    

n_classes = len(np.unique(y))
n_feats = X.shape[1]
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = split_size, random_state = seed)

normalizer = Normalizer().fit(x_train)
encoder = LabelEncoder().fit(y_train)

x_train = normalizer.transform(x_train)
x_test = normalizer.transform(x_test)
y_train = encoder.transform(y_train)
y_test  = encoder.transform(y_test)




In [ ]:
def evaluate_model(model, samples, targets):
    pred = model.predict(samples)
    return accuracy_score(targets, pred)

In [ ]:
model = MODEL_FUNCTION(**model_params)
model.fit(x_train, y_train)

acc_train = evaluate_model(model, x_train, y_train)
acc_test  = evaluate_model(model, x_test, y_test)

print(acc_train, acc_test)

### Interpretability

In [ ]:
def get_shap_feature_importances(model, x_train):
    '''Get scores with shap'''
    samples = np.random.choice(list(range(len(x_train))), min(5, len(x_train)), replace = False)
    se = shap.KernelExplainer(model.predict, x_train[samples])
    shap_values = se.shap_values(x_train)
    importance_order = np.argsort(-abs(np.abs(np.array(shap_values)).mean(axis = 0)))
    importances = np.abs(np.array(shap_values)).mean(axis = 0)[importance_order]
    return importance_order, importances

order, importances = get_shap_feature_importances(model, x_train)

## Testing

In [ ]:
def evaluate_model_feats_removed(x_train, y_train, x_test, y_test, n_feats, feat_indices, k = 4, trials = 15):
    selected_feats = list(set(np.arange(n_feats)).difference(feat_indices[:k]))      # exclude top k features
    x_t = x_train[:, selected_feats]
    x_tst = x_test[:, selected_feats]
    
    train_accs = []
    test_accs = []
    
    for i in range(trials):

        m = MODEL_FUNCTION(**model_params).fit(x_t, y_train)
        acc_train = evaluate_model(m, x_t, y_train)
        acc_test  = evaluate_model(m, x_tst, y_test)
        
        train_accs.append(acc_train)
        test_accs.append(acc_test)

        
    train_accs, test_accs = np.array(train_accs), np.array(test_accs)
    
    return {'train_acc': train_accs.mean(),
            'train_var': train_accs.var(),
            'test_acc': test_accs.mean(),
            'test_var': test_accs.var()}

### Local2Global

In [ ]:
#setup

num_trials = 500

#shap
order_shap, _ = get_shap_feature_importances(model, x_train)

#fit
l2g_exp = Local2GlobalExplainer(x_train, model, n_classes)
mc_exp, _ = l2g_exp.mcmc_estimate(num_trials)
is_exp, _ = l2g_exp.importance_sampling(num_trials)

#order calculation
order_mc = l2g_exp.get_only_feature_importance(mc_exp)
order_is = l2g_exp.get_only_feature_importance(is_exp)

In [ ]:
num_feats_removed_limit = 6
trials = 100

scores = pd.DataFrame()

for rem_feats in range(1, num_feats_removed_limit+1):
    shap_scores =  evaluate_model_feats_removed(x_train, y_train, x_test, y_test, n_feats, order_shap, k = rem_feats, trials = trials)
    s_df = pd.DataFrame(shap_scores, index = [0])
    s_df = s_df.drop(columns = ['train_acc', 'train_var']).rename(columns = {'test_acc': '(SHAP) Test acc', 'test_var': '(SHAP) Test var'})
    
    mc_scores =  evaluate_model_feats_removed(x_train, y_train, x_test, y_test, n_feats, order_mc, k = rem_feats, trials = trials)
    mc_df = pd.DataFrame(mc_scores, index = [0])
    mc_df =mc_df.drop(columns = ['train_acc', 'train_var']).rename(columns = {'test_acc': '(MCMC) Test acc', 'test_var': '(MCMC) Test var'})
    
    
    is_scores =  evaluate_model_feats_removed(x_train, y_train, x_test, y_test, n_feats, order_is, k = rem_feats, trials = trials)
    is_df = pd.DataFrame(is_scores, index = [0])
    is_df = is_df.drop(columns = ['train_acc', 'train_var']).rename(columns = {'test_acc': '(Imp. Samp.) Test acc', 'test_var': '(Imp. Samp.) Test var'})
    
    
    test = pd.concat([s_df, mc_df, is_df], axis = 1)
    test['Features Removed'] = rem_feats
    
    scores = pd.concat([scores, test], axis = 0)

scores = scores.reset_index(drop = True)
scores['Dataset'] = MODEL_NAME

order = list(scores.columns[-2:]) + list(scores.columns[:-2])
scores = scores[order].set_index("Features Removed")
scores

In [ ]:
scores.to_csv(f"Benchmarks/benchmarks_{MODEL_NAME}.csv", index = True)